In [39]:
# from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.chrome.options import Options
import os
import json
import re
import datetime
from selenium.webdriver.common.action_chains import ActionChains
from seleniumwire import webdriver  # Import from seleniumwire
driver = webdriver.Chrome()

chrm_options=Options()
chrm_caps = webdriver.DesiredCapabilities.CHROME.copy()
chrm_caps['goog:loggingPrefs'] = { 'performance':'ALL' } #save all returns including websocket data
driver = webdriver.Chrome(chrome_options=chrm_options,
                          desired_capabilities=chrm_caps) 

# options.add_argument("--disable-popup-blocking")
driver = webdriver.Chrome()

In [40]:
website = 'https://www.tradingview.com/'

In [41]:
driver.get(website)

In [53]:
driver.find_element_by_xpath("//*[text()='Search markets here']").click()

In [54]:
query = driver.find_element_by_name('query')
query.send_keys('BTCN2019')
query.submit()

In [55]:
query.send_keys(Keys.ENTER)

In [56]:
control = driver.find_element_by_id('header-toolbar-intervals')
control.click()

In [57]:
select_1h = driver.find_element_by_xpath("//div[@data-value='60']")
select_1h.click()

In [58]:
driver.refresh()
driver.switch_to_alert().accept()

/Users/francis/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: use driver.switch_to.alert instead
  


In [59]:
def wheel_element(element, deltaX = 0, deltaY = 120, offsetX = 0, offsetY = 0):
    error = element._parent.execute_script("""
    var element = arguments[0];
    var deltaX = arguments[1]
    var deltaY = arguments[2];
    var box = element.getBoundingClientRect();
    var clientX = box.left + (arguments[3] || box.width / 2);
    var clientY = box.top + (arguments[4] || box.height / 2);
    var target = element.ownerDocument.elementFromPoint(clientX, clientY);

    for (var e = target; e; e = e.parentElement) {
      if (e === element) {
        target.dispatchEvent(new MouseEvent('mouseover', {view: window, bubbles: true, cancelable: true, clientX: clientX, clientY: clientY}));
        target.dispatchEvent(new MouseEvent('mousemove', {view: window, bubbles: true, cancelable: true, clientX: clientX, clientY: clientY}));
        target.dispatchEvent(new WheelEvent('wheel',     {view: window, bubbles: true, cancelable: true, clientX: clientX, clientY: clientY, deltaX: deltaX, deltaY: deltaY}));
        return;
      }
    }    
    return "Element is not interactable";
    """, element, deltaX, deltaY, offsetY)
    if error:
        raise WebDriverException(error)

In [60]:
elm = driver.find_element_by_class_name('chart-gui-wrapper')

In [61]:
for i in range(100):
    wheel_element(elm, deltaX=0, deltaY=100)

for i in range(100):
    wheel_element(elm, deltaX=10, deltaY=0)

In [62]:
ws = []
for request in driver.requests:
    if request.ws_messages:
        ws.append(request.ws_messages)

In [63]:
messages = []
for w in ws:
    for l in w:
        m = l.content
        txt_list = re.split('~m~\d*~m~',m)       
        for t in txt_list:
            if len(t)<10:
                continue
            t = json.loads(t)
            try:
                if t['m'] == 'timescale_update':
                    messages.append(t)
            except:
                continue

In [64]:
messages_after_refresh = []
for m in messages:
    try:
        if m['m']=='timescale_update':
            messages_after_refresh.append(m) 
    except:
        continue

In [65]:
price_data = []

for m in messages:
    try:
        _j = m['p'][1]
        price_data.append([s['v']  for s in _j['sds_1']['s']])
    except:
        continue

In [66]:
import pandas as pd
df = [pd.DataFrame(p) for p in price_data]
df = pd.concat(df)

df.columns = ['Date', 'open', 'high', 'low', 'close', 'volume']
df.Date = pd.to_datetime(df.Date, unit='s')

In [67]:
print('before drop duplicate', len(df))
df = df.drop_duplicates('Date')
df.sort_values('Date', inplace=True)
print('after',len(df))
df.reset_index(inplace=True, drop=True)

before drop duplicate 2014
after 1244


In [68]:
from matplotlib import pyplot as plt


plt.figure(figsize=(10,5))
plt.plot(df.Date, df.open)
plt.plot(df.Date, df.high)
plt.plot(df.Date, df.low)
plt.plot(df.Date, df.close)


/Users/francis/opt/anaconda3/lib/python3.7/site-packages/matplotlib/cbook/__init__.py:1402: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  x[:, None]
/Users/francis/opt/anaconda3/lib/python3.7/site-packages/matplotlib/cbook/__init__.py:1402: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  x[:, None]
/Users/francis/opt/anaconda3/lib/python3.7/site-packages/matplotlib/axes/_base.py:276: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  x = x[:, np.newaxis]
/Users/francis/opt/anaconda3/lib/python3.7/site-packages/matplotlib/axes/_base.py:278: FutureWarning: Support for multi-dimensional in

In [70]:
df.head(60)

,Date,open,high,low,close,volume
0,2019-04-25 22:00:00,5080.0,5080.0,5080.0,5080.0,1.0
1,2019-04-28 22:00:00,5195.0,5195.0,5145.0,5145.0,1.0
2,2019-04-29 22:00:00,5245.0,5245.0,5230.0,5230.0,1.0
3,2019-04-30 22:00:00,5280.0,5290.0,5280.0,5290.0,1.0
4,2019-05-01 22:00:00,5295.0,5380.0,5295.0,5380.0,1.0
5,2019-05-02 22:00:00,5500.0,5770.0,5500.0,5670.0,26.0
6,2019-05-05 22:00:00,5745.0,5745.0,5745.0,5745.0,0.0
7,2019-05-06 22:00:00,5870.0,5905.0,5780.0,5835.0,65.0
8,2019-05-07 22:00:00,5715.0,5885.0,5685.0,5880.0,82.0
9,2019-05-08 22:00:00,6010.0,6060.0,5960.0,6050.0,66.0
